## Supplementary material for "A Simple Electrostatic Model for the Hard-Sphere Solute Component of Nonpolar Solvation"
#### Christopher D. Cooper and Jaydeep P. Bardhan

This Jupyter notebook explores results from solvation free energy calculations using an implicit solvent model from the paper "A Simple Electrostatic Model for the Hard-Sphere Solute Component of Nonpolar Solvation", and molecular dynamics from Mobley et al. "Small Molecule Hydration Free Energies in Explicit Solvent: An Extensive Test of Fixed-Charge Atomistic Simulations" (2009). The test set corresponds to 497 small solutes from D. Mobley's paper.

The code reads in the files `implicit_data.txt` and `explicit_data.txt` which are available in the Supplementary Material. These are CSV files with the names and different free energy components for the 497 molecules. 

In [1]:
import numpy
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [6]:

def plot_correl(X, Y, names, key='None', title='Energy', xlabel='X-axis', ylabel='Y-axis'):
    """
    Uses plotly to do scatter plot of correlation between two datasets
    Inputs:
    -------
        X    : (array of float) data in X axis
        Y    : (array of float) data in Y axis
        names: (array of string) molecule names
        key  : (string) highlight points that contain 'key' in the name
        title: (string) title of the plot. Defaults to 'Energy'
        xlabel: (string) x label of the plot
    Output:
    -------
        Plotly scatter plot with hover tags
        Correlation statistics
    """
    key_true = []
    for i in range(len(names)):
        if key in names[i]:
            key_true.append("red")
        else:
            key_true.append("blue")

    trace = go.Scatter(
                x = X,
                y = Y,
                text = names,
                mode='markers',
                marker = dict(size=8,
                            opacity=0.6, color=key_true),
                hoverinfo = 'text')


    max_val = numpy.max(X)
    min_val = numpy.min(X)
    ref = go.Scatter( x = numpy.array([min_val,max_val]),
                      y = numpy.array([min_val,max_val]),
                      mode = 'lines'
                    )

    data = [trace, ref]
    
    layout= go.Layout(
        title= 'Energy',
        hovermode= 'closest',
        xaxis= dict(
            title= 'MD',
            ticklen= 5,
            zeroline= False,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'BEM',
            ticklen= 5,
            gridwidth= 2,
        ),
        showlegend= False
    )
    
    correl = numpy.corrcoef(X, Y)
    rmsd = numpy.sqrt(numpy.sum(numpy.abs(X-Y)**2)/len(X))
    mue = numpy.sum(numpy.abs(X-Y))/len(X)

    print ('Correlation coeff: %1.4f'%correl[0,1])
    print ('RMSD: %1.4f'%rmsd)
    print ('MUE: %1.4f'%mue)
    
    fig= go.Figure(data=data, layout=layout)
    iplot(fig)
    
    
def read_explicit(filename):
    """
    Reads in data of solvation free energy from explicit_data.txt (taken from Mobley et al 2009)
    Input:
    ------
        filename: (string) name of file (probably explicit_data.txt)
        
    Output:
    -------
        iupac: (array of str) names of molecules in IUPAC
        smiles: (array of str) names of molecuiles in SMILES 
        dGes: (array of float) electrostatic component of solvation free energy 
        dGcav: (array of float) free energy to generate the solute-shaped cavity 
        dGdisp: (array of float) free energy of the solute-solvent dispersion interaction 
        dGexper: (array of float) solvation free energy measured experimentally
    """
    
    iupac = []; smiles = []; dGes = []; dGcav = []; dGdisp = []; dGexper = []
    with open(filename) as f:
        line = f.readline()
        line = f.readline()

        while line:
            line_split = line.split()
            iupac.append(line_split[0][:-1])
            smiles.append(line_split[1][:-1])
            dGes.append(float(line_split[2][:-1]))
            dGcav.append(float(line_split[3][:-1]))
            dGdisp.append(float(line_split[4][:-1]))
            dGexper.append(float(line_split[5]))
            line = f.readline()

            
    iupac   = numpy.array(iupac)
    smiles  = numpy.array(smiles)
    dGes    = numpy.array(dGes)
    dGcav   = numpy.array(dGcav)
    dGdisp  = numpy.array(dGdisp)
    dGexper = numpy.array(dGexper)
    
    return iupac, smiles, dGes, dGcav, dGdisp, dGexper
            
    
def read_implicit(filename):
    """
    Reads in data of solvation free energy from implicit_data.txt 
    Input:
    ------
        filename: (string) name of file (probably implicit_data.txt)
        
    Output:
    -------
        iupac: (array of str) names of molecules in IUPAC
        smiles: (array of str) names of molecuiles in SMILES 
        dGes: (array of float) electrostatic component of solvation free energy 
        dGcav: (array of float) free energy to generate the solute-shaped cavity 
        dGdisp: (array of float) free energy of the solute-solvent dispersion interaction 
        dGexper: (array of float) solvation free energy measured experimentally
        sasa: (array of float) solvent accessible surface area 
        volume: (array of float) molecular volume (inside the SAS)
    """
    iupac = []; smiles = []; dGes = []; dGcav = []; dGdisp = []; dGexper = []; sasa = []; vol = [];
    with open(filename) as f:
        line = f.readline()
        line = f.readline()

        while line:
            line_split = line.split()
            iupac.append(line_split[0][:-1])
            smiles.append(line_split[1][:-1])
            dGes.append(float(line_split[2][:-1]))
            dGcav.append(float(line_split[3][:-1]))
            dGdisp.append(float(line_split[4][:-1]))
            dGexper.append(float(line_split[5][:-1]))
            sasa.append(float(line_split[6][:-1]))
            vol.append(float(line_split[7]))
            line = f.readline()

            
    iupac   = numpy.array(iupac)
    smiles  = numpy.array(smiles)
    dGes    = numpy.array(dGes)
    dGcav   = numpy.array(dGcav)
    dGdisp  = numpy.array(dGdisp)
    dGexper = numpy.array(dGexper)
    sasa    = numpy.array(sasa)
    vol     = numpy.array(vol)
    
    return iupac, smiles, dGes, dGcav, dGdisp, dGexper, sasa, vol

Here we read in the files...

In [7]:
iupac, smiles, dGes_i, dGcav_i, dGdisp_i, dGexper, SASA, vol = read_implicit('implicit_data.txt')
iupac, smiles, dGes_e, dGcav_e, dGdisp_e, dGexper = read_explicit('explicit_data.txt')

All the `_i` files are energy components for the implicit solvent model, whereas `_e` for the explicit solvent. `iupac` and `smiles` are the molecule names, which are the same in both files. `SASA` is the solvent accessible surface area and `vol` the volume of each solute.

As a reminder:
* $\Delta G_{cav}$ is the cavity energy
* $\Delta G_{disp}$ is the energy of the solute-solvent dispersion interacion
* $\Delta G_{es}$ is the electrostatic component of free energy (charging free energy)
* $\Delta G_{np} = \Delta G_{cav} + \Delta G_{disp}$ is the nonpolar component of the solvation free energy
* $\Delta G = \Delta G_{np} + \Delta G_{es}$ is the total solvation free energy
* $\Delta G_{exper}$ is the solvation free energy obtained from experiments

Next, you can explore the correlation between explicit and implicit solvent calculations for each energy component, and with the experimental values. You might find the following very useful:
* Hover over the scatter plot, you'll be able to see the name of each solute, given by either IUPAC or SMILES (see docstring under the `plot_correl` function). 
* Highlight in red components with a certain string in its name using the `key` argument. For example, if you want to see all the compounds with triple bonds, your third input argument would be `smiles` and you can add `key='#'`  

In [8]:
dG_i = dGes_i + dGcav_i + dGdisp_i
dG_e = dGes_e + dGcav_e + dGdisp_e
plot_correl(dGexper, dG_i, iupac, key='dimethyl')

Correlation coeff: 0.9256
RMSD: 1.3335
MUE: 1.0213
